# 1: Imports e configurações

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import requests  # se quiser baixar nova planilha no futuro

# Config para exibir mais colunas/linhas no pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# 2: Carregar o CSV gerado

In [3]:
# Baixando a planilha mais recente do GitHub
url = "https://raw.githubusercontent.com/elissouza2023/DasboardExportacoesAco/main/data/raw/Performance-Mensal_2025.12.xls"
response = requests.get(url)
response.raise_for_status()  # Garante que o download deu certo

with open("Performance-Mensal_2025.12.xls", "wb") as f:
    f.write(response.content)

print("Planilha baixada com sucesso!")

# Lendo diretamente o XLS baixado
df_raw = pd.read_excel("Performance-Mensal_2025.12.xls", engine="xlrd", header=None)

# Exibe as primeiras linhas para inspecionar a estrutura
print("Primeiras 20 linhas do XLS bruto:")
print(df_raw.head(20))

# Exibe nomes únicos da coluna 0 (onde estão os nomes das métricas/produtos)
print("\nNomes únicos na coluna 0 (produtos/métricas):")
print(df_raw[0].unique())

Planilha baixada com sucesso!
Primeiras 20 linhas do XLS bruto:
                                                  0         1         2    \
0    Siderurgia Brasileira / Brazilian Steel Industry       NaN       NaN   
1   Comparativo Performance Mensal / Monthly Perfo...       NaN       NaN   
2                                                 NaN       NaN       NaN   
3                        Especificação\nSpecification      2013       NaN   
4                                                 NaN  Jan\nJan  Fev\nFeb   
5                                                 NaN       NaN       NaN   
6                               Produção / Production       NaN       NaN   
7                             Aço Bruto / Crude Steel  2769.437  2574.894   
8                     Laminados / Rolled Products (*)  2051.441  1942.836   
9                              Planos / Flat Products  1202.482  1144.078   
10                             Longos / Long Products   848.959   798.758   
11  Semiacab

# 3: Limpeza e criação de data

In [5]:
# -------------------------------
# Limpeza do Excel bruto (formato largo)
# -------------------------------

# 1. Inspecionar estrutura (já vimos no head(20), mas confirmamos aqui)
print("Shape do DataFrame bruto:", df_raw.shape)
print("\nColunas:", df_raw.columns.tolist()[:15])  # Primeiras colunas

# 2. Definir onde começam os dados reais
# Pelo seu output:
# Linha 3 = cabeçalho "Especificação" + anos
# Linha 4 = meses (Jan Fev Mar ...)
# Linha 7 em diante = métricas + valores

# Pegar a linha de meses (ajuste o índice se necessário)
linha_meses = 4   # "NaN  Jan\nJan  Fev\nFeb  ..."
meses_br = df_raw.iloc[linha_meses, 1:].str.extract(r'([A-Za-z]{3})', expand=False).str.lower().tolist()

# Mapear para números (só os válidos)
mes_map = {'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05', 'jun': '06',
           'jul': '07', 'ago': '08', 'set': '09', 'out': '10', 'nov': '11', 'dez': '12'}
meses_num = [mes_map.get(m, None) for m in meses_br if m in mes_map]

print("\nMeses detectados:", meses_br[:12])  # Deve mostrar jan, fev, mar...
print("Meses numéricos:", meses_num[:12])

# 3. Pegar as linhas de métricas (a partir da linha 7)
df_dados = df_raw.iloc[6:, :].copy()  # Começa em "Produção / Production"

# Renomear a coluna 0 como 'especificacao'
df_dados = df_dados.rename(columns={0: 'especificacao'})

# Filtrar só linhas com nome de métrica (não NaN)
df_dados = df_dados[df_dados['especificacao'].notna() & df_dados['especificacao'].str.strip().ne('')]

# Métricas agregadas que queremos (exatas ou aproximadas - ajuste se necessário)
metricas_desejadas = [
    'Aço Bruto / Crude Steel',
    'Vendas Internas / Domestic Sales (*)',
    'Vendas Externas / Foreign Market (**)',
    'Consumo Aparente / Apparent Consumption (***)'
]

# Filtrar apenas essas linhas
df_filtrado = df_dados[df_dados['especificacao'].isin(metricas_desejadas)].copy()

print("\nMétricas encontradas:")
print(df_filtrado['especificacao'].unique())

# 4. Melt para formato longo
# Colunas de valores começam na coluna 1
df_long = pd.melt(
    df_filtrado,
    id_vars=['especificacao'],
    value_vars=df_filtrado.columns[1:],
    var_name='coluna_original',
    value_name='volume'
)

# Converter volume para numérico
df_long['volume'] = pd.to_numeric(df_long['volume'], errors='coerce')

# Criar ano e mês a partir da posição da coluna
# Como os anos se repetem a cada 12 colunas, calculamos
n_meses_por_ano = 12
col_idx = df_long['coluna_original'].astype(int) - 1  # volta para índice 0-based
df_long['ano'] = 2013 + (col_idx // n_meses_por_ano)
df_long['mes_idx'] = col_idx % n_meses_por_ano
df_long['mes'] = df_long['mes_idx'].map(lambda x: meses_br[x] if x < len(meses_br) else None)

# Criar data
df_long['mes_num'] = df_long['mes'].map(mes_map)
df_long['date'] = pd.to_datetime(
    df_long['ano'].astype(str) + '-' + df_long['mes_num'] + '-01',
    errors='coerce'
)

# Remover linhas inválidas
df_long = df_long.dropna(subset=['date', 'volume'])

# Ordenar
df_long = df_long.sort_values('date').reset_index(drop=True)

# Pivot para formato wide (melhor para dashboard)
df_limpo = df_long.pivot(
    index='date',
    columns='especificacao',
    values='volume'
).reset_index()

# Renomear colunas para uso fácil
df_limpo = df_limpo.rename(columns={
    'Aço Bruto / Crude Steel': 'producao_aco_bruto',
    'Vendas Internas / Domestic Sales (*)': 'vendas_internas',
    'Vendas Externas / Foreign Market (**)': 'exportacoes_volume',
    'Consumo Aparente / Apparent Consumption (***)': 'consumo_aparente'
})

# Calcular importações e saldo
df_limpo['importacoes_volume'] = df_limpo['consumo_aparente'] - df_limpo['vendas_internas']
df_limpo['saldo_comercial_volume'] = df_limpo['exportacoes_volume'] - df_limpo['importacoes_volume']

df_limpo = df_limpo.round(3)

print("\nDataFrame limpo (primeiras linhas):")
print(df_limpo.head(10))

print("\nInfo:")
print(df_limpo.info())

Shape do DataFrame bruto: (54, 158)

Colunas: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

Meses detectados: ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
Meses numéricos: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

Métricas encontradas:
['Aço Bruto / Crude Steel' 'Vendas Internas / Domestic Sales (*)'
 'Vendas Externas / Foreign Market (**)'
 'Consumo Aparente / Apparent Consumption (***)']

DataFrame limpo (primeiras linhas):
especificacao       date  producao_aco_bruto  consumo_aparente  \
0             2013-01-01            2769.437          2167.705   
1             2013-02-01            2574.894          2069.840   
2             2013-03-01            2858.521          2330.914   
3             2013-04-01            2913.622          2402.633   
4             2013-05-01            2999.775          2456.820   
5             2013-06-01            2836.535          2358.624   
6             2013-07-01      

# 4: Salvando os dados

In [8]:
# Salvar o arquivo limpo (use um nome descritivo)
df_limpo.to_csv("dados_siderurgia_limpos_2013_2025.csv", index=False, encoding='utf-8')
print("Arquivo salvo: dados_siderurgia_limpos_2013_2025.csv")

Arquivo salvo: dados_siderurgia_limpos_2013_2025.csv


# 5: Criando gráficos comparativos

In [12]:

# ----------------------------------------------------------------------
# Gráficos de teste (rode para validar os insights)
# ----------------------------------------------------------------------

import plotly.express as px
import plotly.graph_objects as go

# 1. Vendas internas vs Exportações + % exportação
df_melt1 = df_limpo.melt(
    id_vars='date',
    value_vars=['vendas_internas', 'exportacoes_volume'],
    var_name='Tipo',
    value_name='Volume (mil t)'
)

fig1 = px.bar(
    df_melt1,
    x='date',
    y='Volume (mil t)',
    color='Tipo',
    barmode='group',
    title='Vendas Internas vs Exportações – Brasil (2013–2025)'
)

# Linha % exportações
pct_export = (df_limpo['exportacoes_volume'] / (df_limpo['vendas_internas'] + df_limpo['exportacoes_volume'])) * 100
fig1.add_trace(
    go.Scatter(
        x=df_limpo['date'],
        y=pct_export,
        name='% Exportações',
        yaxis='y2',
        line=dict(color='black', width=2, dash='dash')
    )
)

fig1.update_layout(
    yaxis2=dict(title='% Exportações', overlaying='y', side='right', range=[0, 50]),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig1.show()

# 2. Exportações vs Importações + saldo comercial
df_melt2 = df_limpo.melt(
    id_vars='date',
    value_vars=['exportacoes_volume', 'importacoes_volume'],
    var_name='Tipo',
    value_name='Volume (mil t)'
)

fig2 = px.bar(
    df_melt2,
    x='date',
    y='Volume (mil t)',
    color='Tipo',
    barmode='group',
    title='Exportações vs Importações – Balança Comercial do Aço'
)

fig2.add_trace(
    go.Scatter(
        x=df_limpo['date'],
        y=df_limpo['saldo_comercial_volume'],
        name='Saldo Comercial (mil t)',
        yaxis='y2',
        line=dict(color='darkgreen', width=2.5)
    )
)

fig2.update_layout(
    yaxis2=dict(title='Saldo (mil t)', overlaying='y', side='right'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig2.show()

# 3. Consumo aparente vs Vendas internas
df_melt3 = df_limpo.melt(
    id_vars='date',
    value_vars=['consumo_aparente', 'vendas_internas'],
    var_name='Tipo',
    value_name='Volume (mil t)'
)

fig3 = px.bar(
    df_melt3,
    x='date',
    y='Volume (mil t)',
    color='Tipo',
    barmode='group',
    title='Consumo Aparente vs Vendas Internas (pressão de importados)'
)
fig3.show()

KeyError: 'produto'